# Installing packages:
  
pip install numpy

pip install python-opencv 

In [2]:
import numpy as np
import cv2
import math
import datetime


def relativize(m):
    m[m >= 0] = 1+np.log(1+m[m >= 0])
    m[m < 0] = np.exp(m[m < 0])
    return m

def get_frame(cap, color_mode=0):
    ret, frame0 = cap.read()
    if (color_mode == 0):
        frame = frame0
    elif (color_mode == 1):
        frame = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
    elif (color_mode == 2):
        frame = cv2.cvtColor(frame0, cv2.COLOR_BGR2HSV)
    return frame0, frame

def ini_memory(frame):
    avg = np.ones(frame.shape, dtype=float)*128
    err = np.ones(frame.shape, dtype=float)*128
    return avg, err


# ------------- PARAMS ----------------
rate = 0.1     # Update avg with this amount of the new value
power = 1      # Strength of the effect, 0 = None, 1 = ok.
reduce = 2     # Resolution reduction (1, 2 or 4):
max_err = 32   # 10=high sensitivity, 32=noise-free output 
mode = 0       # different ways of showing the results
color_mode = 0 # 0=RGB, 1=grayscxale, 2=HSV 
SaveVideo = False
# --------------------------------------

# Define size of the output images:
width = 640 // reduce
heigh = 480 // reduce

# Open webcam:
cap = cv2.VideoCapture(0)
ret = cap.set(3, width)
ret = cap.set(4, heigh)

# Create avg and stdev arrays, same shape as frame, but float populated:
frame0, frame = get_frame(cap, color_mode)   
avg, err = ini_memory(frame)

# Initialize output images:
output = cv2.copyMakeBorder(frame,0,0,0,0,cv2.BORDER_REPLICATE)  # BGR
triple = cv2.copyMakeBorder(frame,0,0,0,width,cv2.BORDER_REPLICATE)

# Prepare video output file:
if SaveVideo:
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_RT.avi',fourcc, 1.0, (triple.shape[0], triple.shape[1]), True)

fps = 0

while(True):  
    time1 = datetime.datetime.now()

    # Read a new image from source:
    frame0, frame = get_frame(cap, color_mode)   
      
    # Vectorized version:
    if (mode == 0):
        # neutral gray gets darker or lighter with movement:
        coef = relativize((frame - avg) / max_err / 2)
        output = np.clip(126 * np.power(coef, power), 0, 255)
    if (mode == 1):
        # black gets lighter with movement:
        coef = np.power(relativize(np.abs(frame - avg) / max_err), power) - 1   
        output = np.clip(256 * coef, 0, 255)
    if (mode == 2):
        # frame get lighter with movement:
        coef = np.power(relativize(np.abs(frame - avg) / max_err), power) - 1
        output = np.clip(frame * coef, 0, 255)

    # Update average and mean error:
    avg = (1-rate) * avg + rate * frame   
    # max_err = 5*np.sqrt(np.average(np.square(frame - avg)))
            
    # Show original, coefs and then filtered:
    triple[1:frame.shape[0], 1:frame.shape[1]] = frame[1:frame.shape[0], 1:frame.shape[1]]
    triple[1:frame.shape[0], frame.shape[1]+1:frame.shape[1]*2] = output[1:frame.shape[0], 1:frame.shape[1]]   
    # triple[1:frame.shape[0], frame.shape[1]*2+1:frame.shape[1]*3] = mixed[1:frame.shape[0], 1:frame.shape[1]]
       
    # Show final composite image:
    cv2.imshow('Relativize video filter', triple)    
    # write the frame to video:
    if SaveVideo:
        out.write(triple)

    elapsedTime = datetime.datetime.now() - time1
    fps = 1 / elapsedTime.total_seconds()
    print('mode:', mode, 'fps: ', fps, end='   \r')
       
    # Press q to exit:    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('m'):
        mode = divmod(mode + 1, 3)[1]
    if key == ord('c'):
        color_mode = divmod(color_mode + 1, 3)[1]
        frame0, frame = get_frame(cap, color_mode)   
        avg, err = ini_memory(frame)     
        output = cv2.copyMakeBorder(frame,0,0,0,0,cv2.BORDER_REPLICATE)  # BGR
        triple = cv2.copyMakeBorder(frame,0,0,0,width,cv2.BORDER_REPLICATE)
    if key == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
# If video doesnt show, manually reset the kernel to free the file:
if SaveVideo:
    out.release